In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score
from scipy.stats import kstest, mannwhitneyu
import itertools
# import shap

from tqdm import tqdm

import matplotlib.pyplot as plt
import pingouin as pg
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# file_path = "C:/Users/shrra/Downloads/MASH biomarkers clinical data.csv"
file_path = "C:/Users/shrra/Downloads/MASH_biomarkers_clinicaldata_notimputed_May30_HH.csv"

df = pd.read_csv(file_path)
print(df.columns)

df['MASH'] = df['MASH'].map({'MASH': 1, 'non-MASH': 0})

excluded_cols = ["ID", "MASH"]
without_meadian_df = pd.read_csv(file_path)
with_meadian_df = pd.read_csv(file_path)


without_meadian_df['MASH'] = without_meadian_df['MASH'].map({'MASH': 1, 'non-MASH': 0})

Index(['ID', 'MASH', 'Age (yrs.)', 'Female n(%)', 'Hispanic race/ethnicity',
       'BMI (kg/m2)', 'BMI z-score', 'WC (cm)', 'ALT (U/L)', 'AST (U/L)',
       'GGT (U/L)', 'Glucose (mg/dL)', 'Insulin (uU/mL)', 'HOMA-IR',
       'TG:HDL ratio', 'TG (mg/dL)', 'TC (mg/dL)', 'LDL-c (mg/dL)',
       'HDL-c (mg/dL)', 'Bilirubin (mg/dL)', 'Creatinine (mg/dL)',
       'Albumin (g/dL)', 'Alk phos (U/L)', 'Uric acid (mg/dL)',
       'Platelet count', 'SBP', 'DBP'],
      dtype='object')


In [3]:
KSTest_df = without_meadian_df


excluded_cols = ["ID"]
KSTest_df= KSTest_df.drop(columns=excluded_cols)


print(KSTest_df.columns)

Index(['MASH', 'Age (yrs.)', 'Female n(%)', 'Hispanic race/ethnicity',
       'BMI (kg/m2)', 'BMI z-score', 'WC (cm)', 'ALT (U/L)', 'AST (U/L)',
       'GGT (U/L)', 'Glucose (mg/dL)', 'Insulin (uU/mL)', 'HOMA-IR',
       'TG:HDL ratio', 'TG (mg/dL)', 'TC (mg/dL)', 'LDL-c (mg/dL)',
       'HDL-c (mg/dL)', 'Bilirubin (mg/dL)', 'Creatinine (mg/dL)',
       'Albumin (g/dL)', 'Alk phos (U/L)', 'Uric acid (mg/dL)',
       'Platelet count', 'SBP', 'DBP'],
      dtype='object')


dropping na rows

In [4]:
KSTest_df  = KSTest_df.dropna()

Select features which have the least P value below

In [7]:
import pandas as pd
import numpy as np
from scipy.stats import ks_2samp

# Assuming KSTest_df is your DataFrame
# Calculate correlation matrix
corr_matrix = KSTest_df.drop('MASH', axis=1).corr().abs()

# Set threshold for high correlation
corr_threshold_low = 0.5
corr_threshold_high = 1.0

# Dictionary to store highly correlated features for each feature
highly_correlated_features = {}

# Loop through the features and find highly correlated features
for feature in corr_matrix.columns:
    highly_corr = corr_matrix[(corr_matrix[feature] > corr_threshold_low) & (corr_matrix[feature] < corr_threshold_high)][feature].index.tolist()
    if highly_corr:
        highly_correlated_features[feature] = highly_corr

# Print the highly correlated features for each feature
for feature, correlated in highly_correlated_features.items():
    print(f"Highly correlated features for {feature}: {correlated}")

# Function to calculate p-value using KS test
def ks_test(feature, target):
    return ks_2samp(feature[target == 0], feature[target == 1]).pvalue

# Perform KS test for each group and keep the feature with the best p-value
selected_features = set()
ks_test_results = {}

for feature, correlated in highly_correlated_features.items():
    p_values = {col: ks_test(KSTest_df[col], KSTest_df['MASH']) for col in correlated + [feature]}
    ks_test_results[feature] = p_values
    best_feature = min(p_values, key=p_values.get)
    selected_features.add(best_feature)

# Print KS test results and selected feature from each group
print("\nKS Test Results and Selected Features from Each Group:\n")
for feature, p_values in ks_test_results.items():
    best_feature = min(p_values, key=p_values.get)
    print(f"KS test p-values for group starting with {feature}: {p_values}, Best Feature: {best_feature}")

# Final set of features
final_selected_features = list(selected_features)

# Print the selected features
print("\nSelected Features:\n", final_selected_features)


Highly correlated features for Age (yrs.): ['WC (cm)', 'Creatinine (mg/dL)', 'Alk phos (U/L)']
Highly correlated features for BMI (kg/m2): ['BMI z-score', 'WC (cm)']
Highly correlated features for BMI z-score: ['BMI (kg/m2)']
Highly correlated features for WC (cm): ['Age (yrs.)', 'BMI (kg/m2)']
Highly correlated features for ALT (U/L): ['AST (U/L)', 'GGT (U/L)']
Highly correlated features for AST (U/L): ['ALT (U/L)']
Highly correlated features for GGT (U/L): ['ALT (U/L)']
Highly correlated features for Insulin (uU/mL): ['HOMA-IR']
Highly correlated features for HOMA-IR: ['Insulin (uU/mL)']
Highly correlated features for TG:HDL ratio: ['TG (mg/dL)', 'HDL-c (mg/dL)']
Highly correlated features for TG (mg/dL): ['TG:HDL ratio', 'TC (mg/dL)']
Highly correlated features for TC (mg/dL): ['TG (mg/dL)', 'LDL-c (mg/dL)']
Highly correlated features for LDL-c (mg/dL): ['TC (mg/dL)']
Highly correlated features for HDL-c (mg/dL): ['TG:HDL ratio']
Highly correlated features for Creatinine (mg/dL): ['

Select p values which has less than 0.05 below

In [8]:
import pandas as pd
import numpy as np
from scipy.stats import ks_2samp

# Assuming KSTest_df is your DataFrame
# Calculate correlation matrix
corr_matrix = KSTest_df.drop('MASH', axis=1).corr().abs()

# Set threshold for high correlation
corr_threshold_low = 0.5
corr_threshold_high = 1.0

# Dictionary to store highly correlated features for each feature
highly_correlated_features = {}

# Loop through the features and find highly correlated features
for feature in corr_matrix.columns:
    highly_corr = corr_matrix[(corr_matrix[feature] > corr_threshold_low) & (corr_matrix[feature] < corr_threshold_high)][feature].index.tolist()
    if highly_corr:
        highly_correlated_features[feature] = highly_corr

# Print the highly correlated features for each feature
for feature, correlated in highly_correlated_features.items():
    print(f"Highly correlated features for {feature}: {correlated}")

# Function to calculate p-value using KS test
def ks_test(feature, target):
    return ks_2samp(feature[target == 0], feature[target == 1]).pvalue

# Perform KS test for each group and keep the feature with the best p-value
selected_features = set()
ks_test_results = {}

for feature, correlated in highly_correlated_features.items():
    p_values = {col: ks_test(KSTest_df[col], KSTest_df['MASH']) for col in correlated + [feature]}
    ks_test_results[feature] = p_values
    best_feature = min(p_values, key=p_values.get)
    if p_values[best_feature] < 0.05:
        selected_features.add(best_feature)

# Print KS test results and selected feature from each group
print("\nKS Test Results and Selected Features from Each Group:\n")
for feature, p_values in ks_test_results.items():
    best_feature = min(p_values, key=p_values.get)
    print(f"KS test p-values for group starting with {feature}: {p_values}, Best Feature: {best_feature} (p-value: {p_values[best_feature]})")

# Final set of features
final_selected_features = list(selected_features)

# Print the selected features
print("\nSelected Features with p-value < 0.05:\n", final_selected_features)


Highly correlated features for Age (yrs.): ['WC (cm)', 'Creatinine (mg/dL)', 'Alk phos (U/L)']
Highly correlated features for BMI (kg/m2): ['BMI z-score', 'WC (cm)']
Highly correlated features for BMI z-score: ['BMI (kg/m2)']
Highly correlated features for WC (cm): ['Age (yrs.)', 'BMI (kg/m2)']
Highly correlated features for ALT (U/L): ['AST (U/L)', 'GGT (U/L)']
Highly correlated features for AST (U/L): ['ALT (U/L)']
Highly correlated features for GGT (U/L): ['ALT (U/L)']
Highly correlated features for Insulin (uU/mL): ['HOMA-IR']
Highly correlated features for HOMA-IR: ['Insulin (uU/mL)']
Highly correlated features for TG:HDL ratio: ['TG (mg/dL)', 'HDL-c (mg/dL)']
Highly correlated features for TG (mg/dL): ['TG:HDL ratio', 'TC (mg/dL)']
Highly correlated features for TC (mg/dL): ['TG (mg/dL)', 'LDL-c (mg/dL)']
Highly correlated features for LDL-c (mg/dL): ['TC (mg/dL)']
Highly correlated features for HDL-c (mg/dL): ['TG:HDL ratio']
Highly correlated features for Creatinine (mg/dL): ['